In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [134]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [135]:
cat = Table.read('./Catalog/nsa_20hosts_sample_211103_visual_inspect.fits')

In [136]:
junk = (cat['bad_votes'] > cat['good_votes'])
candy = (cat['is_candy'] > cat['bad_votes'])
gal = (~junk) & (~candy)

In [137]:
print('Candy:', np.sum(candy))
print('Gal:', np.sum(gal))
print('Junk:', np.sum(junk))

Candy: 162
Gal: 111
Junk: 21


In [138]:
cat = cat[gal] # junks only
len(cat)

111

In [139]:
cat.sort('viz-id')

### Generate RGB images

In [140]:
def _display_cutout_rgb(images, ax=None, half_width=200):
    import scarlet
    from scarlet.display import AsinhMapping

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    
    # Crop
    if half_width is not None:
        cen = (images.shape[1] // 2, images.shape[2] // 2)
        images = images[:, cen[0] - half_width:cen[0]+half_width, cen[1] - half_width:cen[1] + half_width]
        
    # Display
    norm = AsinhMapping(minimum=-0.2, stretch=1, Q=1)
    img_rgb = scarlet.display.img_to_rgb(images, norm=norm)
    plt.imshow(img_rgb, origin='lower')
    ax.axis('off')
    
    if ax is None:
        return fig
    return ax

In [141]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/20hosts/gal/'

In [142]:
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")

In [143]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'grizy'])
        fig, ax = plt.subplots(figsize=(5, 5))
        ax = _display_cutout_rgb(images, ax=ax, half_width=200)
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 111
Progress: 10 / 111
Progress: 20 / 111
Progress: 30 / 111
Progress: 40 / 111
Progress: 50 / 111
Progress: 60 / 111
Progress: 70 / 111
Progress: 80 / 111
Progress: 90 / 111
Progress: 100 / 111
Progress: 110 / 111


In [144]:
# Copy figures in FIGURE_DIR to WEBPAGE_DIR
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        if not os.path.isdir(WEBPAGE_DIR):
            os.mkdir(WEBPAGE_DIR)
            
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

### Generate webpage for cutouts. WE use `diezi/webpage/gen_web_cutout.py`

In [145]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [146]:
#FOR NSA SAMPLE
FIGURE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/20hosts/gal/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/20hosts/gal/'
SCARLET_DIR = '/tigress/jiaxuanl/public_html/NSA/vanilla_result/'
sample_name = 'NSA'

In [147]:
webpage_cutout(FIGURE_DIR, WEBPAGE_DIR, sample_name, sample_title='Galaxy', SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 111 galaxies to be displayed
Total pages: 5


In [64]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/20hosts/


### The search box on webpage still doesn't work. Need to work on the javascript file